<a href="https://colab.research.google.com/github/dhcryan/dm_project/blob/main/dm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import metrics
from sklearn.cluster import *
from scipy.spatial.distance import cdist
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D

In [61]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u342-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [62]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import re
import sys
import itertools as it
from pyspark import SparkConf, SparkContext

In [65]:
import pyspark
from pyspark.sql import SQLContext

In [66]:
file_path='/content/drive/MyDrive'
fifa_path='/players_fifa23.csv'

import pyspark 
from pyspark.sql import SparkSession 


In [67]:
spark=SparkSession.builder.appName('fifa').getOrCreate()

In [68]:
# # File location and type
# file_location = "/content/drive/MyDrive/players_fifa23.csv"
# file_type = "csv"

# # CSV options
# infer_schema = "true"
# first_row_is_header = "true"
# delimiter = ","

# # The applied options are for CSV files. For other file types, these will be ignored.
# df = spark.read.format(file_type) \
#   .option("inferSchema", infer_schema) \
#   .option("header", first_row_is_header) \
#   .option("sep", delimiter) \
#   .load(file_location)

# df.show(10)

In [69]:
# # The applied options are for CSV files. For other file types, these will be ignored.
df=spark.read.option("inferSchema", "true").option("header", "true").csv('/content/drive/MyDrive/players_fifa23.csv')
df.show(10) 

+--------+-----------------------+------------------------------------+----+-------+-------+-------------------------------------------------+-------------------------+--------+----------+-------+-----------+----------+--------------+-------------+--------------------------------------+----------+--------+--------------+-------------+--------------+-----------+-----------+-------+--------------------+-----------------+---------------+--------------+--------------+---------+-----------+------------------+------------------+----------+--------------+-------------+---------------+---------------+-----------------+---------+----------+----------------+-------------+--------+----------+------+-----------+------------+------------+-------------+------------+--------+----------+--------+----------+--------+--------+---------+----------+-----------+--------------+------------+-------+----------+----------+--------+---------------+--------------+---------+-----------+----------+--------------+-

In [70]:
from pyspark.sql.functions import trim
#strip header space
for each in df.schema.names:
    df = df.withColumnRenamed(each,re.sub(r'\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*','',each.replace(' ', '')))
df.show(10)

+--------+--------------------+--------------------+----+------+------+--------------------+--------------------+-------+---------+------+----------+---------+--------------+-------------+--------------------+--------+--------+-------------+-------------+--------------+-----------+----------+-------+--------------------+-----------------+---------------+--------------+-------------+--------+----------+------------------+------------------+---------+-------------+------------+--------------+--------------+----------------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------+--------+--------+--------+--------+--------+---------+--------+--------+--------+---------+---------+---

In [71]:
from pyspark.sql.functions import *

for c_name in df.columns:
    if c_name in ['PhotoUrl','Nationality','Positions','BestPosition','ClubPosition','ContractUntil','ClubNumber','OnLoad','NationalTeam','NationalPosition','NationalNumber','PreferredFoot','AttackingWorkRate','DefensiveWorkRate']:
      df = df.withColumn(c_name, trim(col(c_name)))
df.show()

+--------+--------------------+--------------------+----+------+------+--------------------+--------------+-------+---------+------+----------+---------+---------+------------+--------------------+--------+--------+-------------+------------+-------------+----------+----------+------+------------+----------------+--------------+-------------+-------------+--------+----------+-----------------+-----------------+---------+-------------+------------+--------------+--------------+----------------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------+--------+--------+--------+--------+--------+---------+--------+--------+--------+---------+---------+---------+--------+--------+----

### missing value analysis


In [72]:
# Find count for empty, None, Null, Nan with string literals.
from pyspark.sql.functions import col,isnan,when,count
missing_df = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
missing_df.show()

# ClubPosition|ContractUntil|ClubNumber 92개의 missing value
# NationalPosition|NationalNumber -> 국대 아닌 선수들 17722
# 그 외의 결측치 없음

+---+----+--------+---+------+------+--------+-----------+-------+---------+------+----------+---------+---------+------------+----+--------+-------+-------------+------------+-------------+----------+----------+------+------------+----------------+--------------+-------------+-------------+--------+----------+-----------------+-----------------+---------+-------------+------------+--------------+--------------+----------------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------+--------+--------+--------+--------+--------+---------+--------+--------+--------+---------+---------+---------+--------+--------+--------+--------+
| ID|Name|FullName|Age|Height|Weight|PhotoUrl|Natio

In [73]:
df.printSchema()

root
 |-- ID: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- PhotoUrl: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Overall: double (nullable = true)
 |-- Potential: double (nullable = true)
 |-- Growth: double (nullable = true)
 |-- TotalStats: double (nullable = true)
 |-- BaseStats: double (nullable = true)
 |-- Positions: string (nullable = true)
 |-- BestPosition: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- ValueEUR: double (nullable = true)
 |-- WageEUR: double (nullable = true)
 |-- ReleaseClause: double (nullable = true)
 |-- ClubPosition: string (nullable = true)
 |-- ContractUntil: string (nullable = true)
 |-- ClubNumber: string (nullable = true)
 |-- ClubJoined: double (nullable = true)
 |-- OnLoad: string (nullable = true)
 |-- NationalTeam: string (nullabl

In [74]:
from pyspark.sql.types import IntegerType
df=df.withColumn("ID",df["ID"].cast(IntegerType()))
df=df.withColumn("Age",df["Age"].cast(IntegerType()))
df=df.withColumn("Overall",df["Overall"].cast(IntegerType()))
df=df.withColumn("Potential",df["Potential"].cast(IntegerType()))
df=df.withColumn("IntReputation",df["IntReputation"].cast(IntegerType()))
df=df.withColumn("SkillMoves",df["SkillMoves"].cast(IntegerType()))
df=df.withColumn("WeakFoot",df["WeakFoot"].cast(IntegerType()))
df=df.withColumn("Height",df["Height"].cast(IntegerType()))
df=df.withColumn("Weight",df["Weight"].cast(IntegerType()))


In [75]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- PhotoUrl: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Growth: double (nullable = true)
 |-- TotalStats: double (nullable = true)
 |-- BaseStats: double (nullable = true)
 |-- Positions: string (nullable = true)
 |-- BestPosition: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- ValueEUR: double (nullable = true)
 |-- WageEUR: double (nullable = true)
 |-- ReleaseClause: double (nullable = true)
 |-- ClubPosition: string (nullable = true)
 |-- ContractUntil: string (nullable = true)
 |-- ClubNumber: string (nullable = true)
 |-- ClubJoined: double (nullable = true)
 |-- OnLoad: string (nullable = true)
 |-- NationalTeam: string (n

### attribute 분류

In [76]:
print(df.show())

+------+--------------------+--------------------+---+------+------+--------------------+--------------+-------+---------+------+----------+---------+---------+------------+--------------------+--------+--------+-------------+------------+-------------+----------+----------+------+------------+----------------+--------------+-------------+-------------+--------+----------+-----------------+-----------------+---------+-------------+------------+--------------+--------------+----------------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------+--------+--------+--------+--------+--------+---------+--------+--------+--------+---------+---------+---------+--------+--------+-------

In [77]:
Fifa_df = df.toPandas()
print(Fifa_df)

           ID                     Name                              FullName  \
0      158023  L. Messi                 Lionel Messi                           
1      165153  K. Benzema               Karim Benzema                          
2      188545  R. Lewandowski           Robert Lewandowski                     
3      192985  K. De Bruyne             Kevin De Bruyne                        
4      231747  K. Mbappé                Kylian Mbappé                          
...       ...                      ...                                   ...   
18534  243725  D. Collins               Darren Collins                         
18535  261933  Yang Dejiang             Dejiang Yang                           
18536  267823  L. Mullan                Liam Mullan                            
18537  267824  D. McCallion             Daithí McCallion                       
18538  261424  N. Rabha                 Nabin Rabha                            

       Age  Height  Weight             

In [78]:
Fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18539 entries, 0 to 18538
Data columns (total 90 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 18539 non-null  int32  
 1   Name               18539 non-null  object 
 2   FullName           18539 non-null  object 
 3   Age                18539 non-null  int32  
 4   Height             18539 non-null  int32  
 5   Weight             18539 non-null  int32  
 6   PhotoUrl           18539 non-null  object 
 7   Nationality        18539 non-null  object 
 8   Overall            18539 non-null  int32  
 9   Potential          18539 non-null  int32  
 10  Growth             18539 non-null  float64
 11  TotalStats         18539 non-null  float64
 12  BaseStats          18539 non-null  float64
 13  Positions          18539 non-null  object 
 14  BestPosition       18539 non-null  object 
 15  Club               18539 non-null  object 
 16  ValueEUR           185

## Clustering


In [79]:
#gk와 non gk구분 
def segment_gk(Fifa_df):
  gk_data=Fifa_df[Fifa_df.BestPosition=='GK']
  nongk_data=Fifa_df[Fifa_df.BestPosition!='GK']
  return gk_data,nongk_data
  

In [80]:
gk,non_gk=segment_gk(Fifa_df)


In [81]:
# attrs=['Crossing','Finishing','HeadingAccuracy','ShortPassing','Volleys','Dribbling','Curve'\
#        ,'FKAccuracy','LongPassing','BallControl','Acceleration','SprintSpeed','Agility','Reactions'\
#        ,'Balance','ShotPower','Jumping','Stamina','Strength','LongShots','Aggression','Interceptions'\
#        ,'Positioning','Vision','Penalties','Composure','Marking','StandingTackle','SlidingTackle'\
#        ,'GKDiving','GKHandling','GKKicking','GKPositioning','GKReflexes']
def cluster_attrs(nongk):    
    attrs_groups = dict()
    attrs_groups['passing'] = ['Crossing','ShortPassing','FKAccuracy','LongPassing','Vision']
    attrs_groups['shooting'] = ['Finishing','LongShots','Penalties','ShotPower','Volleys','Curve']
    attrs_groups['pace'] = ['Acceleration','SprintSpeed']
    attrs_groups['skill'] = ['BallControl','Dribbling']
    attrs_groups['movement']=['Agility','Reactions','Positioning','Composure']
    attrs_groups['defending'] = ['Interceptions','Marking','StandingTackle','SlidingTackle']
    attrs_groups['pysicality'] = ['Aggression','Stamina','Strength','Balance','Jumping','HeadingAccuracy']
    #attrs_groups['goalkeeping'] = ['GKDiving', 'GKHandling','GKKicking', 'GKPositioning', 'GKReflexes']
    pass_index=nongk[attrs_groups['passing']].mean(axis=1)
    shoot_index=nongk[attrs_groups['shooting']].mean(axis=1)
    pace_index=nongk[attrs_groups['pace']].mean(axis=1)
    skill_index=nongk[attrs_groups['skill']].mean(axis=1)
    movement_index=nongk[attrs_groups['movement']].mean(axis=1)
    defense_index=nongk[attrs_groups['defending']].mean(axis=1)
    pysical_index=nongk[attrs_groups['pysicality']].mean(axis=1)
    
    index_concat=pd.concat([pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index],axis=1,ignore_index=True)
    #radar chart만들기 위해 normalize해야 할 필요성 있음 min-max normalization
    nongk_value = index_concat.values #returns a numpy array
    scaler = preprocessing.StandardScaler()
    nongk_scaled=scaler.fit_transform(nongk_value)
    return nongk_scaled.round(2)

In [82]:
nongk_scaled=cluster_attrs(non_gk)
nongk_df= pd.DataFrame(nongk_scaled)
nongk_df.columns=['pass_index','shoot_index','pace_index','skill_index','movement_index','defense_index','pysical_index']

코드 실행 후 아래와 같이 표준화된 결과를 얻을 수 있다. 

In [83]:
nongk_df.head()

,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index
0,3.43,2.80,1.20,3.15,3.67,-1.22,0.77
1,2.45,2.67,1.02,2.65,3.09,-1.15,2.02
2,2.24,2.85,0.65,2.45,3.09,-0.85,2.82
3,3.52,2.80,0.56,2.65,2.86,0.67,1.20
4,2.14,2.58,2.62,2.95,3.49,-1.07,1.76


After fitting a PCA object to the standardized matrix, we can see how much of the variance is explained by each of the nine features.

In [84]:
from sklearn.decomposition import PCA
pca=PCA()
pca.fit(nongk_scaled)
evr = pca.explained_variance_ratio_
print(evr)

[0.53406225 0.2333198  0.10916886 0.07671527 0.02135    0.01443702
 0.01094679]


In general, it is a good rule of thumb to preserve around 80% of the variance. Therefore, in this instance, we can select the 3 most important principal components to incorporate in the k-means algorithm.

PCA를 단계적으로 진행할 경우, 공분산 행렬을 통해 고윳값(eigenvalue)과 고유벡터(eigenvector)를 구할 수 있다. 
이때, 고윳값이 설명 가능한 분산량에 해당한다. 
위의 그래프는 주성분 각각의 고윳값을 고윳값들을 모두 더한 값으로 나눠 준 것이다.
즉, 전체에서 해당 주성분의 고윳값이 차지하는 비율을 알아보는 것이다

In [85]:
plt.plot(range(1,len(nongk_df.columns)+1),np.cumsum(evr),marker='o',linestyle='--')
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

원래 dimension이 7이었는데
위 소스 코드의 핵심은 PCA의 파라미터 주성분 개수(n_components)를 결정하여 개수에 맞는 원래 데이터에서 변환된 주성분을 추출하는 것이다. 본 글에서는 주성분을 4개로 결정하였다. (분산이 제일 큰 놈 4개: 데이터가 널리 퍼진거)

This can be accomplished by instantiating a new PCA object with the n_components parameter set to 3.

The explained variance, or eigenvalue, in PCA shows the variance that can be attributed to each of the principal components.

In [86]:
for i, exp_var in enumerate(evr.cumsum()):
  print(i+1,"번째 분산량",evr[i], '\t','누적 분산량: ',exp_var)
  if exp_var >= 0.95:
      n_comps = i + 1
      break
print("Number of components:", n_comps)
# print(sum(evr,[]))
print(pca.explained_variance_ratio_)
pca = PCA(n_components=n_comps)
pca.fit(nongk_scaled)
nongk_scaled_pca = pca.transform(nongk_scaled)

1 번째 분산량 0.5340622533591339 	 누적 분산량:  0.5340622533591339
2 번째 분산량 0.23331979918533788 	 누적 분산량:  0.7673820525444718
3 번째 분산량 0.10916885512772596 	 누적 분산량:  0.8765509076721978
4 번째 분산량 0.07671527456626721 	 누적 분산량:  0.953266182238465
Number of components: 4
[0.53406225 0.2333198  0.10916886 0.07671527 0.02135    0.01443702
 0.01094679]


### Elbow Method for optimal value of k in KMeans

Finding the elbow point of the WCSS (within cluster sum of squares) curve using the kneed KneeLocator

In [87]:
!pip install kneed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
# Cluster 간의 거리의 합을 나타내는 inertia가 급격히 떨어지는 구간이 생기는데
# 이 지점의 K 값을 군집의 개수로 사용
# inertia_속성으로 확인할 수 있다.
# 클러스터 간 거리의 합을 나타내는 inertia value가 급격히 떨어지는 구간 이용
from kneed import KneeLocator
inertia_arr=[]
def elbow_point_for_k(nongk_scaled_pca):
  global inertia_arr
  k_range=range(2,15)
  for k in k_range:
    kmeans_pca=KMeans(n_clusters=k).fit(nongk_scaled_pca)
    inertia=kmeans_pca.inertia_
    inertia_arr.append(inertia)
    print('k value: ',k,'\t','cost: ',inertia)
  n_clusters = KneeLocator([i for i in k_range], inertia_arr, curve='convex', direction='decreasing').knee
  print("Optimal number of clusters", n_clusters)

In [89]:
elbow_point_for_k(nongk_scaled_pca)


k value:  2 	 cost:  70202.50768033802
k value:  3 	 cost:  55468.402449544505
k value:  4 	 cost:  46321.9579982967
k value:  5 	 cost:  41050.64764840132
k value:  6 	 cost:  37751.665076718586
k value:  7 	 cost:  34909.04186850913
k value:  8 	 cost:  32448.41348655395
k value:  9 	 cost:  30244.977009223363
k value:  10 	 cost:  28485.016098215317
k value:  11 	 cost:  27021.249401666017
k value:  12 	 cost:  25786.748518997018
k value:  13 	 cost:  24708.76078468495
k value:  14 	 cost:  23701.609232198898
Optimal number of clusters 5


In [90]:
inertia_arr
print(type(inertia_arr))

<class 'list'>


In [91]:
fig=plt.figure()
ax=fig.add_subplot(111)
ax.plot(range(2,15),inertia_arr,'b*-',linewidth=2)
plt.grid(True)
# plt.vlines(KneeLocator([i for i in range(2,15)], inertia_arr, curve='convex', direction='decreasing').knee, ymin=min(inertia_arr), ymax=max(inertia_arr), linestyles='dashed')
plt.title('Elbow method for KMeans Clustering')
plt.xlabel('Number of Cluster')
plt.ylabel('Average within-cluster sum-of-squares(inertia)')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Average within-cluster sum-of-squares(inertia)')

Finding the elbow point of the WCSS (within cluster sum of squares) curve using the YellowBrick KElbowVisualizer

In [92]:
from yellowbrick.cluster import KElbowVisualizer

In [93]:
visualizer = KElbowVisualizer(KMeans(init='k-means++', random_state=42), k=(2,15), timings=False)
visualizer.fit(nongk_scaled_pca)
visualizer.show()
n_clusters = visualizer.elbow_value_
print("Optimal number of clusters:", n_clusters)

Optimal number of clusters: 5


In [94]:
number_of_clusters = 5
kmeanModel = KMeans(n_clusters = number_of_clusters ) 
kmeanModel.fit(nongk_scaled_pca)

KMeans(n_clusters=5)

In [95]:
# nongk_rescaled['Name']=non_gk['Name']
# nongk_rescaled['Overall']=non_gk['Overall']
# nongk_rescaled['BestPosition']=non_gk['BestPosition']
# nongk_rescaled.insert(0,'Name',non_gk['Name'])
# nongk_rescaled.insert(1,'Club',non_gk['Club'])
# nongk_rescaled['Cluster']=kmeanModel.labels_

In [96]:
type(nongk_scaled_pca)

numpy.ndarray

In [97]:
labels=kmeanModel.labels_

In [98]:
nongk_pca_kmeans_df = pd.concat([nongk_df.reset_index(drop=True), pd.DataFrame(nongk_scaled_pca)], axis=1)
nongk_pca_kmeans_df.columns.values[(-1*n_comps):] = ["PCA Component " + str(i+1) for i in range(n_comps)]
nongk_pca_kmeans_df['Cluster'] = labels
nongk_pca_kmeans_df['Name']=non_gk['Name'].reset_index(drop=True)
nongk_pca_kmeans_df['BestPosition']=non_gk['BestPosition'].reset_index(drop=True)
nongk_pca_kmeans_df

,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
0,3.43,2.80,1.20,3.15,3.67,-1.22,0.77,6.726875,0.342237,0.615166,0.181881,1,L. Messi,CAM
1,2.45,2.67,1.02,2.65,3.09,-1.15,2.02,5.816280,-0.336014,0.132669,1.394052,1,K. Benzema,CF
2,2.24,2.85,0.65,2.45,3.09,-0.85,2.82,5.701664,-1.078143,0.189942,1.999844,1,R. Lewandowski,ST
3,3.52,2.80,0.56,2.65,2.86,0.67,1.20,5.878852,-1.407581,0.854530,-0.417365,1,K. De Bruyne,CM
4,2.14,2.58,2.62,2.95,3.49,-1.07,1.76,6.356858,0.249218,-1.293110,0.964180,1,K. Mbappé,ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16473,-1.16,-0.33,0.01,-1.33,-1.28,-1.25,-3.39,-2.358325,3.163213,0.651042,-1.205391,2,D. Collins,CAM
16474,-1.57,-1.29,-1.27,-1.83,-1.57,-0.16,-1.93,-3.583433,1.092952,0.832385,-0.595490,4,Yang Dejiang,CDM
16475,-0.73,-0.77,-0.36,-1.13,-1.40,-0.81,-2.52,-2.331546,2.086866,0.671840,-1.082719,2,L. Mullan,RM
16476,-3.06,-1.98,-1.50,-3.57,-2.75,0.10,-1.83,-6.079749,1.002441,0.339134,-0.071356,4,D. McCallion,CB


In [99]:
# nongk_pca_kclustered = nongk_scaled_pca
# # nongk_clustered['Cluster']=labels
# nongk_pca_kclustered['Name']=non_gk['Name'].reset_index(drop=True)
# nongk_pca_kclustered['BestPosition']=non_gk['BestPosition'].reset_index(drop=True)
# nongk_pca_kclustered.columns.values[(-1*n_comps):] = ["Component " + str(i+1) for i in range(n_comps)]
# nongk_pca_kclustered['Cluster'] = labels
# nongk_pca_kclustered.head()
# # nongk_clustered.insert(1,'BestPosition',non_gk['BestPosition']).reset_index(drop=True)
# # nongk_clustered.insert(2,'Cluster',labels).reset_index(drop=True)


In [100]:
# nongk_clustered['Cluster']=labels
# nongk_clustered[nongk_clustered['BestPosition'].isnull()]

In [101]:
nongk_pca_kmeans_df[nongk_pca_kmeans_df['Cluster']==4]

,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
317,-0.83,-1.88,-0.54,-0.03,-1.11,1.77,1.55,-1.858921,-2.398845,-0.879955,-0.017618,4,R. Le Normand,CB
444,-1.06,-2.03,0.10,-0.73,-1.28,1.59,1.50,-2.283705,-2.035180,-1.544230,0.070163,4,Unai Núñez,CB
628,-0.19,-1.18,-3.28,-0.63,-0.45,1.54,0.59,-2.055438,-2.491215,2.097914,0.000549,4,G. Maripán,CB
738,-1.30,-1.91,0.51,-1.48,-0.71,1.41,2.61,-2.140516,-2.442256,-2.279435,1.117277,4,J. Aidoo,CB
792,-0.36,-1.29,-1.45,-0.63,-0.91,1.55,0.63,-1.922670,-1.960656,0.432855,-0.291907,4,Domingos Duarte,CB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16470,-1.14,-1.26,-1.22,-1.48,-1.19,-0.17,-1.36,-2.918646,0.676999,0.754083,-0.386302,4,Liao Wei,CDM
16472,-1.06,-1.25,-0.45,-1.58,-1.48,-0.65,-1.41,-2.838487,1.265349,0.176290,-0.331666,4,Pi Ziyang,CAM
16474,-1.57,-1.29,-1.27,-1.83,-1.57,-0.16,-1.93,-3.583433,1.092952,0.832385,-0.595490,4,Yang Dejiang,CDM
16476,-3.06,-1.98,-1.50,-3.57,-2.75,0.10,-1.83,-6.079749,1.002441,0.339134,-0.071356,4,D. McCallion,CB


In [102]:
# kmeans clustering의 문제점: 선수 능력치가 낮은 애들은 그냥 선수특성이고 나발이고 낮은 능력치 애들로 clustering됨

def show_cluster_samples(df, cluster_no, number_of_samples):
    return  df[df['Cluster'] == cluster_no][:number_of_samples]
for cluster_no in range(number_of_clusters):
    display(show_cluster_samples(nongk_pca_kmeans_df, cluster_no, 20))

,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
29,-0.24,-1.31,1.15,0.86,0.19,2.06,1.53,0.173179,-2.192184,-2.055017,-0.601848,0,K. Koulibaly,CB
47,-0.30,-0.78,0.88,0.71,0.36,2.05,2.28,0.438924,-2.643929,-1.898528,0.153084,0,M. Škriniar,CB
51,0.62,-0.40,0.15,-0.18,-0.13,1.92,1.20,0.018890,-2.197707,-0.660209,-0.526339,0,N. Süle,CB
87,-0.20,-1.39,-0.40,-0.53,0.13,2.01,1.83,-0.915996,-2.808926,-0.886914,0.068638,0,S. Savić,CB
92,0.48,-0.64,-0.08,0.71,0.50,2.02,1.81,0.609609,-2.739119,-0.705890,-0.224567,0,S. de Vrij,CB
102,-0.03,0.05,-0.26,-0.63,-0.07,1.96,1.95,-0.262966,-2.626102,-0.567875,0.458913,0,G. Chiellini,CB
125,0.11,-1.06,1.52,0.11,0.91,1.95,2.23,0.645135,-2.505649,-2.476043,-0.037771,0,F. Tomori,CB
147,-0.34,-0.40,0.47,0.11,0.79,1.91,2.35,0.428040,-2.671275,-1.496038,0.573154,0,C. Romero,CB
152,0.54,-0.41,0.70,-0.18,0.70,1.79,2.09,0.671140,-2.520466,-1.421214,0.132855,0,R. Araujo,CB
156,-0.05,-0.61,0.19,-0.23,-0.42,1.94,2.11,-0.411475,-2.649227,-1.164368,0.258992,0,J. Giménez,CB


,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
0,3.43,2.80,1.20,3.15,3.67,-1.22,0.77,6.726875,0.342237,0.615166,0.181881,1,L. Messi,CAM
1,2.45,2.67,1.02,2.65,3.09,-1.15,2.02,5.816280,-0.336014,0.132669,1.394052,1,K. Benzema,CF
2,2.24,2.85,0.65,2.45,3.09,-0.85,2.82,5.701664,-1.078143,0.189942,1.999844,1,R. Lewandowski,ST
3,3.52,2.80,0.56,2.65,2.86,0.67,1.20,5.878852,-1.407581,0.854530,-0.417365,1,K. De Bruyne,CM
4,2.14,2.58,2.62,2.95,3.49,-1.07,1.76,6.356858,0.249218,-1.293110,0.964180,1,K. Mbappé,ST
5,2.31,2.70,1.98,2.65,3.52,-0.40,1.45,6.095835,-0.232463,-0.670677,0.525524,1,M. Salah,RW
6,2.22,2.93,1.15,2.40,3.35,-1.36,2.02,5.887637,-0.088558,0.040198,1.684961,1,Cristiano Ronaldo,ST
7,1.49,0.70,1.02,1.06,1.22,2.21,2.30,2.563735,-2.884141,-1.184262,-0.216847,1,V. van Dijk,CB
8,2.43,2.92,-0.03,2.10,2.92,-0.52,2.32,5.290194,-1.217865,0.931740,1.592610,1,H. Kane,ST
9,2.92,2.60,1.70,3.20,3.32,-1.02,0.44,6.323095,0.627980,0.082362,-0.131042,1,Neymar Jr,LW


,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
3406,0.09,1.69,-3.24,0.66,0.19,-1.12,-1.53,0.242570,1.007407,3.936845,0.420762,2,R. Santa Cruz,ST
3579,-0.97,0.94,-1.54,0.31,0.50,-1.42,0.49,0.156907,0.540790,1.421077,1.930479,2,L. Pfeiffer,ST
3657,-1.08,0.72,-0.08,0.26,-0.19,-2.13,0.70,0.107245,1.351983,0.099337,2.142336,2,D. Dike,ST
3992,-1.22,1.21,-1.00,-0.28,1.39,-1.61,-0.35,0.355259,1.374019,1.155297,1.663378,2,Simy,ST
4021,-0.40,1.11,-1.45,-0.28,0.24,-1.46,0.47,0.103027,0.536712,1.507844,1.829305,2,M. Đurić,ST
4088,-1.79,0.71,-1.36,0.51,0.22,-1.58,1.06,-0.223144,0.488325,0.884269,2.566642,2,A. La Mantia,ST
4138,-0.69,1.40,-1.64,0.61,-0.10,-1.19,-0.14,0.212966,0.781375,1.900249,1.328875,2,A. Cerri,ST
4289,-0.61,0.99,-0.81,0.21,-0.10,-1.08,0.30,0.139658,0.598624,0.912660,1.352874,2,J. Serra,ST
4399,-1.01,0.49,0.38,-0.23,0.04,-1.52,0.40,-0.058422,1.178357,-0.380506,1.537791,2,H. Veerman,ST
4448,-1.32,0.43,-0.77,0.21,0.07,-1.66,0.42,-0.288960,0.984580,0.569450,1.860698,2,A. Albers,ST


,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
43,2.14,2.01,2.16,2.50,2.75,-1.49,0.14,5.183788,1.353528,-0.570915,0.033943,3,K. Coman,LM
49,1.57,1.84,2.44,2.80,2.66,-1.48,0.70,5.100893,1.171002,-1.133879,0.477200,3,Vinícius Jr.,LW
56,1.71,2.39,1.70,1.71,2.52,-1.02,0.63,4.578802,0.730321,-0.393504,0.557888,3,P. Aubameyang,ST
59,2.33,2.42,1.20,2.40,2.80,-1.25,0.59,5.229523,0.656379,0.272855,0.436107,3,Iago Aspas,ST
71,1.16,2.15,0.88,2.15,2.60,-1.32,1.48,4.398712,0.252876,-0.054930,1.520217,3,P. Schick,ST
77,2.57,2.06,1.66,2.20,2.57,-0.72,-0.31,4.914814,0.880991,0.004949,-0.703141,3,A. Gómez,CAM
78,2.31,2.19,0.88,2.55,2.40,-0.94,-0.73,4.692281,1.144395,0.832265,-0.672813,3,D. Mertens,CF
90,2.78,2.30,1.43,2.75,2.40,-1.14,-0.89,5.192009,1.471713,0.577694,-0.945875,3,L. Insigne,CAM
99,2.53,2.35,1.34,2.55,2.55,-1.15,0.28,5.223096,0.783053,0.270503,0.018462,3,E. Hazard,LW
106,1.40,1.24,2.30,2.40,2.23,-0.40,-0.14,4.101817,0.850470,-1.133349,-0.700480,3,M. Diaby,RM


,pass_index,shoot_index,pace_index,skill_index,movement_index,defense_index,pysical_index,PCA Component 1,PCA Component 2,PCA Component 3,PCA Component 4,Cluster,Name,BestPosition
317,-0.83,-1.88,-0.54,-0.03,-1.11,1.77,1.55,-1.858921,-2.398845,-0.879955,-0.017618,4,R. Le Normand,CB
444,-1.06,-2.03,0.10,-0.73,-1.28,1.59,1.50,-2.283705,-2.035180,-1.544230,0.070163,4,Unai Núñez,CB
628,-0.19,-1.18,-3.28,-0.63,-0.45,1.54,0.59,-2.055438,-2.491215,2.097914,0.000549,4,G. Maripán,CB
738,-1.30,-1.91,0.51,-1.48,-0.71,1.41,2.61,-2.140516,-2.442256,-2.279435,1.117277,4,J. Aidoo,CB
792,-0.36,-1.29,-1.45,-0.63,-0.91,1.55,0.63,-1.922670,-1.960656,0.432855,-0.291907,4,Domingos Duarte,CB
834,-1.10,-0.51,-1.68,-0.28,-0.71,1.54,0.70,-1.680437,-1.826788,0.687902,0.229379,4,G. Pezzella,CB
900,0.05,-1.35,-1.00,-1.08,-1.05,1.54,0.63,-1.932366,-1.903462,0.095786,-0.471723,4,D. Zagadou,CB
918,-0.85,-1.51,-1.68,-0.43,-1.28,1.47,0.87,-2.353688,-2.038958,0.420143,0.024801,4,A. Disasi,CB
933,-1.24,-1.21,-0.22,-1.53,-0.42,1.55,1.81,-1.989117,-2.189402,-1.228440,0.804870,4,Vitor Hugo,CB
1011,-0.13,-1.31,-2.19,-0.28,-1.08,1.52,0.70,-1.923523,-2.227045,1.140046,-0.249944,4,Germán,CB


What is 3D PCA?
Most of the time, a PCA plot is a 2D scatter plot in which the data is plotted with two most descriptive principal components. However, you can choose to plot with three PCs instead, and this will create a 3D scatter plot, also called 3D PCA. 

The key difference between 2D PCA and 3D PCA is the number of principal components being selected for plotting. In PCA, principal components are constructed to capture the most variation in the dataset: PC1 describes the most variation, PC2 describes the second most variation, and so forth. As a result, the first two or three PCs can capture most of the variation and the rest can be discarded without losing much information. This can be seen in a scree plot.

In [103]:
type(nongk_pca_kmeans_df['Cluster'])

pandas.core.series.Series

In [104]:
x = nongk_pca_kmeans_df['PCA Component 2']
y = nongk_pca_kmeans_df['PCA Component 1']
fig = plt.figure(figsize=(10, 8))
sns.scatterplot(x, y, hue=nongk_pca_kmeans_df['Cluster'], palette = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:cyan'])
plt.title('Clusters by PCA Components', fontsize=20)
plt.xlabel("Component 2", fontsize=18)
plt.ylabel("Component 1", fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [105]:
# Xax = nongk_pca_kmeans_df['PCA Component 1']
# Yax = nongk_pca_kmeans_df['PCA Component 2']
# Zax = nongk_pca_kmeans_df['PCA Component 3']
# y=labels


# cdict = {0:'red',1:'green',2:'royalblue',3:'violet',4:'gold'}
# labl = {0:'Cluster1',1:'Cluster2',2:'Cluster3',3:'Cluster4',4:'Cluster5'}
# marker = {0:'*',1:'o',2:'H',3:'d',4:'v'}
# alpha = {0:.3, 1:.5,2:.2,3:.4,4:.6}

# fig = plt.figure(figsize=(7,5))
# ax = fig.add_subplot(111, projection='3d')

# fig.patch.set_facecolor('white')
# for l in np.unique(y):
#  ix=np.where(y==l)
#  ax.scatter(Xax[ix], Yax[ix], Zax[ix], c=cdict[l], s=40,
#            label=labl[l], marker=marker[l], alpha=alpha[l])
# # for loop ends
# ax.set_xlabel("First Principal Component", fontsize=14)
# ax.set_ylabel("Second Principal Component", fontsize=14)
# ax.set_zlabel("Third Principal Component", fontsize=14)

# ax.legend()
# plt.show()

In [106]:
Xax = np.array(nongk_pca_kmeans_df['PCA Component 1'])
Yax = np.array(nongk_pca_kmeans_df['PCA Component 2'])
Zax = np.array(nongk_pca_kmeans_df['PCA Component 3'])
y=labels
# cdict = {0:'red',1:'green',2:'royalblue',3:'violet',4:'gold'}
# labl = {0:'Cluster1',1:'Cluster2',2:'Cluster3',3:'Cluster4',4:'Cluster5'}
# marker = {0:'*',1:'o',2:'H',3:'d',4:'v'}
# alpha = {0:.3, 1:.5,2:.2,3:.4,4:.6}

fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111, projection='3d')
fig.patch.set_facecolor('white')

ax.grid(b = True, color ='grey',
        linestyle ='-.', linewidth = 0.3,
        alpha = 0.2)
my_cmap = plt.get_cmap('hsv')
pca_clustered_sctt=ax.scatter(Xax,Yax,Zax,alpha=0.8, marker="^",c=nongk_pca_kmeans_df["Cluster"],s=40,cmap=my_cmap)
ax.set_xlabel("First Principal Component", fontsize=10)
ax.set_ylabel("Second Principal Component", fontsize=10)
ax.set_zlabel("Third Principal Component", fontsize=10)

# fig.colorbar(pca_clustered_sctt, ax = ax, shrink = 0.5, aspect = 5)
ax.legend()
plt.show()

<IPython.core.display.Javascript object>

In [107]:
non_gk['Cluster'] = nongk_pca_kmeans_df['Cluster'].reset_index(drop=True)
non_gk

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Name,FullName,Age,Height,Weight,PhotoUrl,Nationality,Overall,Potential,...,CMRating,RMRating,LWBRating,CDMRating,RWBRating,LBRating,CBRating,RBRating,GKRating,Cluster
0,158023,L. Messi,Lionel Messi,35,169,67,https://cdn.sofifa.net/players/158/023/23_60.png,Argentina,91,91,...,88.0,91.0,67.0,66.0,67.0,62.0,53.0,62.0,22.0,1.0
1,165153,K. Benzema,Karim Benzema,34,185,81,https://cdn.sofifa.net/players/165/153/23_60.png,France,91,91,...,84.0,89.0,67.0,67.0,67.0,63.0,58.0,63.0,21.0,1.0
2,188545,R. Lewandowski,Robert Lewandowski,33,185,81,https://cdn.sofifa.net/players/188/545/23_60.png,Poland,91,91,...,83.0,86.0,67.0,69.0,67.0,64.0,63.0,64.0,22.0,1.0
3,192985,K. De Bruyne,Kevin De Bruyne,31,181,70,https://cdn.sofifa.net/players/192/985/23_60.png,Belgium,91,91,...,91.0,91.0,82.0,82.0,82.0,78.0,72.0,78.0,24.0,1.0
4,231747,K. Mbappé,Kylian Mbappé,23,182,73,https://cdn.sofifa.net/players/231/747/23_60.png,France,91,95,...,84.0,92.0,70.0,66.0,70.0,66.0,57.0,66.0,21.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18534,243725,D. Collins,Darren Collins,21,174,68,https://cdn.sofifa.net/players/243/725/23_60.png,Republic of Ireland,47,56,...,44.0,50.0,41.0,38.0,41.0,40.0,36.0,40.0,15.0,NaN
18535,261933,Yang Dejiang,Dejiang Yang,17,175,60,https://cdn.sofifa.net/players/261/933/23_60.png,China PR,47,57,...,45.0,45.0,47.0,48.0,47.0,49.0,49.0,49.0,15.0,NaN
18536,267823,L. Mullan,Liam Mullan,18,170,65,https://cdn.sofifa.net/players/267/823/23_60.png,Northern Ireland,47,67,...,49.0,52.0,46.0,44.0,46.0,46.0,42.0,46.0,17.0,NaN
18537,267824,D. McCallion,Daithí McCallion,17,178,65,https://cdn.sofifa.net/players/267/824/23_60.png,Republic of Ireland,47,61,...,33.0,33.0,44.0,42.0,44.0,47.0,49.0,47.0,15.0,NaN
